Project Part 3: A Deep Learning Model
In this third and final part of the project you will train a deep learning model on your dataset. Note that the best way to do this will likely be to fine-tune an existing deep learning model such as GPT-2, BERT, etc. This is the same as what you will do in Assign 5, except that rather than using the Airline Tweet dataset you will be using your own dataset. Note that it is also possible to train a deep learning model from scratch with either PyTorch or TensorFlow/Keras, but that in the real world it will be more likely that you will want to leverage the cutting edge performance of a pre-trained deep learning model such as those available through huggingface. 

As with Parts 1 and 2, this should be done in a Jupyter notebook and you should add the notebook to the repository where Assign 1 and 2 are. When you are done, you will then get the URL of your project_part3.ipynb notebook in your GitHub repository, and submit that URL here in Canvas. 

Note: Be sure that the output from each cell in you your notebook files is saved to the .ipynb file before your final commit and upload to your GitHub repository.

 

If you are working with a more challenging NLP task such as text summarization or text generation, then Part 3 should be where you fine tune an existing deep learning model for your task (ex: our notebook on text generation w/ GPT-2 fine tuned on Hemingway Links to an external site.). Whereas, Part 2 should be where you used a pre-trained model out of the box without fine tuning it on your dataset (ex: our notebook on text generation w/ a pre-trained GPT-2 model Links to an external site.). 

In [ ]:
# import all of the python modules/packages you'll need here
import pandas as pd

books= pd.read_csv('books_clean.csv')
books.head(5)